In [57]:
import torchtext
import pkuseg
import pandas as pd

In [58]:
totalpath = '../data/movie_remiew/train_data.csv'
testpath = '../data/movie_remiew/test_data.csv'

In [59]:
totaldf = pd.read_csv(totalpath)
totaldf.head(3)

,label,comment
0,0,国王的工作就是读几句稿子啊
1,0,小朋友看嫌复杂大朋友看想快进的尴尬人物似曾相识唯一的泪点也是复制黏贴
2,0,一个非常丰富传奇的故事拍得这么浅薄大家是怎么给出五星的好奇


In [60]:
totaldf.label.value_counts()

1    24960
0    19108
Name: label, dtype: int64

In [61]:
totaldf.shape[0]

44068

In [62]:
totaldf = totaldf.sample(frac=1).reset_index(drop=True)

In [63]:
totaldf.head()

,label,comment
0,0,我不待见它和我觉得阿甘正传很一般一样
1,0,再怎么温情也掩饰不了现实的黑暗所以我只给一星就像素媛妈妈说的要是世界上每个孩子都这样就好了不...
2,1,萤火虫极致的美更加反衬出战争极致的丑妹妹的角色好真实就像每个小女孩幼时那般天真烂漫可惜最后的...
3,0,麻烦你告诉我那点惊悚了
4,1,欢乐温馨有笑有泪朴实无华纯正范儿的早期公路动画三个好基友一生一起走外加一个单身狗大雾


In [64]:
validdf = totaldf.iloc[:4000]
traindf = totaldf.iloc[4000:]

validdf.to_csv('../data/movie_remiew/validset.csv', index=False)
traindf.to_csv('../data/movie_remiew/trainset.csv', index=False)

In [65]:
testdf = pd.read_csv(testpath)
testdf.head(3)

,comment
0,画面很美但台词实在是太太太矫情不是十几岁少年的正常青涩哪怕故作的沧桑而像没文化的油腻中年对键...
1,这片子好看好看好看好看重要的问题问四遍竟然是前百失望致幻的嗨药是弱者逃避现实的选择你觉得现实...
2,老电影有老电影说不出来的美感即使是近三个小时的电影细节做的仍非精致七武士的故事人类百科


In [66]:
testdf.shape[0]

9999

In [67]:
seg = pkuseg.pkuseg()

In [68]:
TEXT = torchtext.data.Field(init_token='<sos>', eos_token='<eos>', tokenize=seg.cut, fix_length=10)
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

In [69]:
train, valid = torchtext.data.TabularDataset.splits(path='../data/movie_remiew/', 
                                                    train='trainset.csv', 
                                                    validation='validset.csv',
                                                    format='csv', 
                                                    skip_header=True,
                                                    fields=[('label', LABEL), 
                                                            ('text', TEXT),])

In [70]:
TEXT.build_vocab(train)

In [84]:
TEXT.vocab.stoi['我']

7

In [85]:
train[0].text

['没有',
 '刻意',
 '的',
 '友情',
 '和',
 '爱情',
 '衣着',
 '也',
 '破旧',
 '就是',
 '有',
 '一',
 '种',
 '自然',
 '朴素',
 '天成',
 '的',
 '感觉']

In [86]:
train[0].label

'1'

In [87]:
trainiter, validiter =  torchtext.data.Iterator.splits((train, valid), batch_sizes=(64, 64))